# Advent of Code 2025

In [38]:
abstract class Problem(private val filename: String) {
    abstract val testResult: String
    fun run() {
        val testOutput = solve("testData/$filename")
        val actualOutput = solve("data/$filename")
        println("Test results: Expected=$testResult Actual=$testOutput")
        println("Solution: $actualOutput")
    }
    abstract fun solve(path: String): String
}

## Day 1: Secret Entrance
### Part 1

In [5]:
import java.io.File
class DayOnePartOne(override val testResult: String = "3") : Problem("day1.txt") {
    override fun solve(path: String): String {
        var current = 50
        var zeroCount = 0
        File(path).useLines { lines ->
            lines.forEach { line ->
                val dir = line.first()
                val count = line.drop(1).toInt()

                if (dir == 'R') {
                    current += count
                } else {
                    current -= count
                }

                if (current % 100 == 0) {
                    zeroCount++
                }
            }
        }
        return zeroCount.toString()
    }
}

DayOnePartOne().run()

Test results: Expected=3 Actual=3
Solution: 1139


### Part 2

In [10]:
class DayOnePartTwo(override val testResult: String = "6") : Problem("day1.txt") {
    override fun solve(path: String): String {
        var current = 50
        var zeroCount = 0
        File(path).useLines { lines ->
            lines.forEach { line ->
                val direction = line.first()
                val count = line.drop(1).toInt()

                if (direction == 'R') {
                    current += count % 100
                    if (current > 100) {
                        zeroCount++
                        current = current - 100
                    }
                    zeroCount += count / 100
                } else {
                    val prevWasZero = current == 0
                    current -= count % 100
                    if (current < 0) {
                        if (!prevWasZero) {
                            zeroCount++
                        }
                        current = 100 + current
                    }
                    zeroCount += count / 100
                }

                if (current % 100 == 0) {
                    zeroCount++
                    current = 0
                }
            }
        }
        return zeroCount.toString()
    }
}

DayOnePartTwo().run()

Test results: Expected=6 Actual=6
Solution: 6684


## Day 2: Invalid IDs
### Part 1

In [13]:
class DayTwoPartOne(override val testResult: String = "1227775554") : Problem("day2.txt") {
    override fun solve(path: String): String {
        var acc = 0L
        File(path).useLines { lines ->
            lines.first().split(",").map { it.split("-") }.forEach { split ->
                for (i in split[0].toLong()..split[1].toLong()) {
                    val str = i.toString()
                    if (str.length % 2 == 0 && str.take(str.length / 2) == str.takeLast(str.length / 2)) {
                        acc += i
                    }
                }
            }
        }
        return acc.toString()
    }
}

DayTwoPartOne().run()

Test results: Expected=1227775554 Actual=1227775554
Solution: 23534117921


### Part 2

Number is 123123
1. Check if 123 = 123 -> bail


Number is 121212
1. Check if 121 = 212
2. Check if 12 = 12 = 12 -> bail

Number is 999
1. If all numbers are the same -> bail

In [43]:
class DayTwoPartTwo(override val testResult: String = "4174379265") : Problem("day2.txt") {
    override fun solve(path: String): String {
        var acc = 0L
        File(path).useLines { lines ->
            lines.first().split(",").map { it.split("-") }.forEach { split ->
                for (i in split[0].toLong()..split[1].toLong()) {
                    if (i < 10) continue
                    val str = i.toString()
                    if (str.all { it == str.first() }) {
                        println("All numbers are equal: $i")
                        acc += i
                        continue
                    }

                    for (j in (str.length / 2) downTo 2) {
                        val chunked = str.chunked(j)
                        if (chunked.size > 1 && chunked.all { it == chunked.first() }) {
                            println("Chunky boi found: $i")
                            acc += i
                            break
                        }
                    }
                }
            }
        }
        return acc.toString()
    }
}

DayTwoPartTwo().run()

All numbers are equal: 11
All numbers are equal: 22
All numbers are equal: 99
All numbers are equal: 111
All numbers are equal: 999
Chunky boi found: 1010
Chunky boi found: 1188511885
All numbers are equal: 222222
Chunky boi found: 446446
Chunky boi found: 38593859
Chunky boi found: 565656
Chunky boi found: 824824824
Chunky boi found: 2121212121
Chunky boi found: 4545
Chunky boi found: 4646
Chunky boi found: 4747
Chunky boi found: 4848
Chunky boi found: 4949
Chunky boi found: 5050
Chunky boi found: 5151
Chunky boi found: 5252
Chunky boi found: 5353
Chunky boi found: 5454
All numbers are equal: 5555
Chunky boi found: 5656
Chunky boi found: 5757
Chunky boi found: 5858
Chunky boi found: 5959
Chunky boi found: 6060
Chunky boi found: 6161
Chunky boi found: 6262
Chunky boi found: 6363
Chunky boi found: 6464
Chunky boi found: 6565
All numbers are equal: 6666
Chunky boi found: 6767
Chunky boi found: 6868
Chunky boi found: 6969
Chunky boi found: 7070
Chunky boi found: 7171
Chunky boi found: 727